In [1]:
import os
import datetime
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input

In [3]:
base_model = MobileNet(weights='imagenet', include_top=False) 

/home/marcolini/anaconda3/envs/deep_myq/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [4]:
DATASET_DIR = '../myq_dataset/Selfie-dataset/labelled/'

In [43]:
images = os.listdir(DATASET_DIR)

images = pd.DataFrame(images)
images = images.rename(index=int, columns={0: "filename"})

In [46]:
images.head()

,filename,negative,neutral,positive,male,female,baby,child,teenager,youth,middle_age,senior
0,2_1_3_925473_240060952784357_1812030612_a.jpg,0,0,1,0,1,0,0,0,1,0,0
1,2_1_3_1171037_236837476520319_348230056_a.jpg,0,0,1,0,1,0,0,0,1,0,0
2,2_1_3_10261202_614305691993395_506992923_a.jpg,0,0,1,0,1,0,0,0,1,0,0
3,1_1_3_10013334_555648927867451_503825726_a.jpg,0,1,0,0,1,0,0,0,1,0,0
4,1_1_3_10261211_754796697894552_907528579_a.jpg,0,1,0,0,1,0,0,0,1,0,0


In [45]:
images['negative'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '0')))
images['neutral'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '1')))
images['positive'] = images['filename'].apply(lambda x: (int(x.split('_')[0] == '2')))

images['male'] = images['filename'].apply(lambda x: (int(x.split('_')[1] == '0')))
images['female'] = images['filename'].apply(lambda x: (int(x.split('_')[1] == '1')))

images['baby'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '0')))
images['child'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '1')))
images['teenager'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '2')))
images['youth'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '3')))
images['middle_age'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '4')))
images['senior'] = images['filename'].apply(lambda x: (int(x.split('_')[2] == '5')))

In [49]:
images.sum()

filename      2_1_3_925473_240060952784357_1812030612_a.jpg2...
negative                                                   3124
neutral                                                   11136
positive                                                   7900
male                                                       6001
female                                                    16159
baby                                                         53
child                                                       395
teenager                                                   2678
youth                                                     18237
middle_age                                                  787
senior                                                       10
dtype: object

In [56]:
X_train, X_test, y_train, y_test = train_test_split(images['filename'], images[images.columns[1:]]) 

In [63]:
images.columns[1:]

Index(['negative', 'neutral', 'positive', 'male', 'female', 'baby', 'child',
       'teenager', 'youth', 'middle_age', 'senior'],
      dtype='object')

In [86]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [87]:
columns = images.columns[1:].tolist()

In [88]:
train_generator = train_datagen.flow_from_dataframe(dataframe=images[:16000], 
                                                    directory=DATASET_DIR, 
                                                    x_col='filename', 
                                                    y_col=columns,
                                                    class_mode='other',
                                                    target_size=(224,224),
                                                    color_mode='rgb')

test_generator = test_datagen.flow_from_dataframe(dataframe=images[16000:], 
                                                    directory=DATASET_DIR, 
                                                    x_col='filename', 
                                                    y_col=columns,
                                                    class_mode=None,
                                                    target_size=(224,224),
                                                    color_mode='rgb')

Found 16000 images.
Found 6163 images.


In [85]:
next(train_generator)[1]

array([[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0,

In [91]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x) 

output_layers = []
for i in range(11):
    output_layers.append(Dense(1, activation = 'sigmoid')(x))

model = Model(inputs=base_model.input, outputs=output_layers)

In [92]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 3 864         conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalizationV1) (None, None, None, 3 128         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

In [16]:
opt = tf.train.AdamOptimizer()

In [17]:
model.compile(optimizer=opt, loss=['binary_crossentropy']*11, metrics=['accuracy'])

In [18]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x, [batch_y[:,i] for i in range(11)])

In [113]:
next(generator_wrapper(train_generator))[1]

AttributeError: 'list' object has no attribute 'shape'

In [114]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_test = test_generator.n//test_generator.batch_size

In [20]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit_generator(generator=generator_wrapper(train_generator),
                    steps_per_epoch=step_size_train,
                    validation_data=(generator_wrapper(test_generator)),
                    validation_steps=step_size_test,
                    callbacks=[tensorboard_callback],
                    verbose=0,
                    epochs=10)

Instructions for updating:
Use tf.cast instead.
  8/192 [>.............................] - ETA: 10:42 - loss: 4.2288 - dense_3_loss: 0.5080 - dense_4_loss: 0.6518 - dense_5_loss: 0.6280 - dense_6_loss: 0.6412 - dense_7_loss: 0.6670 - dense_8_loss: 0.0311 - dense_9_loss: 0.0858 - dense_10_loss: 0.4043 - dense_11_loss: 0.4627 - dense_12_loss: 0.1109 - dense_13_loss: 0.0380 - dense_3_acc: 0.8672 - dense_4_acc: 0.5859 - dense_5_acc: 0.6484 - dense_6_acc: 0.7969 - dense_7_acc: 0.7930 - dense_8_acc: 0.9961 - dense_9_acc: 0.9844 - dense_10_acc: 0.8906 - dense_11_acc: 0.8398 - dense_12_acc: 0.9766 - dense_13_acc: 0.9961   

In [ ]:
model.save_weights('weights/mobilenet', save_format='tf')

In [ ]:
images = os.listdir(DATASET_DIR)

In [ ]:
im = Image.open(os.path.join(DATASET_DIR, images[0]))

In [ ]:
plt.imshow(im)

In [ ]:
img = image.load_img(os.path.join(DATASET_DIR, images[0]), target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

In [ ]:
print('Predicted:', decode_predictions(preds, top=5)[0])